In [5]:
import pysam
import pandas as pd
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
# from model import db, Patient, Variant, Gene, patient_has_variant

In [ ]:
vcf = pysam.VariantFile("/home/marti/MBHS/DBW/OncoTrack/vcf_files/HNSC_sample_muts.hg38.vcf")
variants = []
for record in vcf:
    chrom = record.chrom  # Cromosoma
    pos = record.pos  # Posición
    ref = record.ref  # Alelo de referencia
    alt = ','.join(str(a) for a in record.alts)  # Alelos alternativos
    
    if len(ref) > 1 or any(len(a) > 1 for a in record.alts):
        ref = ref[1:] if len(ref) > 1 else "-"
        alt = alt[1:] if len(alt) > 1 else "-"

    variants.append({
        "chrom": chrom,
        "pos": pos,
        "ref": ref,
        "alt": alt
    })

In [13]:
for variant in variants:
    print(variant)

{'chrom': '1', 'pos': 1013497, 'ref': 'G', 'alt': 'A'}
{'chrom': '1', 'pos': 1014154, 'ref': 'C', 'alt': 'T'}
{'chrom': '1', 'pos': 3767395, 'ref': 'C', 'alt': 'T'}
{'chrom': '1', 'pos': 6154802, 'ref': 'G', 'alt': 'C'}
{'chrom': '1', 'pos': 7736529, 'ref': 'C', 'alt': 'T'}
{'chrom': '1', 'pos': 9932850, 'ref': 'G', 'alt': 'C'}
{'chrom': '1', 'pos': 9981135, 'ref': 'C', 'alt': 'G'}
{'chrom': '1', 'pos': 11073936, 'ref': 'C', 'alt': 'G'}
{'chrom': '1', 'pos': 11228799, 'ref': 'G', 'alt': 'A'}
{'chrom': '1', 'pos': 11803440, 'ref': 'G', 'alt': 'A'}
{'chrom': '1', 'pos': 12321915, 'ref': 'C', 'alt': 'T'}
{'chrom': '1', 'pos': 12825218, 'ref': 'G', 'alt': 'A'}
{'chrom': '1', 'pos': 15932794, 'ref': 'C', 'alt': 'A'}
{'chrom': '1', 'pos': 15933641, 'ref': 'C', 'alt': 'T'}
{'chrom': '1', 'pos': 15933707, 'ref': 'C', 'alt': 'G'}
{'chrom': '1', 'pos': 15933746, 'ref': 'C', 'alt': 'G'}
{'chrom': '1', 'pos': 16052213, 'ref': 'C', 'alt': 'T'}
{'chrom': '1', 'pos': 16207745, 'ref': 'C', 'alt': 'G'}

In [4]:
cosmic_df = pd.read_csv("/home/marti/MBHS/DBW/OncoTrack/reduced_cosmic_mutations.tsv", sep='\t')
# print(cosmic_df.head())

/tmp/ipykernel_8842/1627996770.py:1: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  cosmic_df = pd.read_csv("/home/marti/MBHS/DBW/OncoTrack/reduced_cosmic_mutations.tsv", sep='\t')


In [30]:
matched_variants = []

for variant in variants:
    chrom = variant["chrom"]
    pos = variant["pos"]
    ref = variant["ref"]
    alt = variant["alt"]
        
# Filtrar las variantes en COSMIC que coincidan en cromosoma y posición
    matching_rows = cosmic_df[(cosmic_df['CHROMOSOME'] == chrom) &
                              (cosmic_df['GENOME_START'] <= pos) &
                              (cosmic_df['GENOME_STOP'] == pos)]
        
    for _, row in matching_rows.iterrows():
# Aquí comparamos más detalles si es necesario, por ejemplo, ref/alt o cualquier otro campo
        matched_variants.append({
            "chrom": chrom,
            "pos": pos,
            "ref": ref,
            "alt": alt,"cosmic_id": row["GENOMIC_MUTATION_ID"],
            "mutation_desc": row["MUTATION_DESCRIPTION"]
            })

for variant in matched_variants:
    print(variant)

{'chrom': '1', 'pos': 114713907, 'ref': 'T', 'alt': 'A', 'cosmic_id': 'COSV54736991', 'mutation_desc': 'missense_variant'}
{'chrom': '1', 'pos': 114713907, 'ref': 'T', 'alt': 'A', 'cosmic_id': 'COSV54736320', 'mutation_desc': 'missense_variant'}
{'chrom': '1', 'pos': 114713907, 'ref': 'T', 'alt': 'A', 'cosmic_id': 'COSV54744813', 'mutation_desc': 'synonymous_variant'}
{'chrom': '1', 'pos': 114713907, 'ref': 'T', 'alt': 'A', 'cosmic_id': 'COSV54758732', 'mutation_desc': 'missense_variant'}
{'chrom': '17', 'pos': 7675994, 'ref': 'C', 'alt': 'T', 'cosmic_id': 'COSV52718605', 'mutation_desc': 'splice_region_variant,synonymous_variant'}
{'chrom': '17', 'pos': 7675994, 'ref': 'C', 'alt': 'T', 'cosmic_id': 'COSV52665709', 'mutation_desc': 'splice_region_variant,synonymous_variant'}
{'chrom': '17', 'pos': 7675994, 'ref': 'C', 'alt': 'T', 'cosmic_id': 'COSV105874107', 'mutation_desc': 'intron_variant,splice_donor_variant'}
{'chrom': '17', 'pos': 7675994, 'ref': 'C', 'alt': 'T', 'cosmic_id': 'CO